In [1]:
!pwd
!cd ../ && pwd && ls
!pwd

/Users/adriansamoticha/Desktop/train_deepgaze_3/jupyter_notebook
/Users/adriansamoticha/Desktop/train_deepgaze_3
jupyter_notebook
/Users/adriansamoticha/Desktop/train_deepgaze_3/jupyter_notebook


In [2]:
!pip install git+https://github.com/matthias-k/DeepGaze

  Cloning https://github.com/matthias-k/DeepGaze to /private/var/folders/kd/_s6tmg_d3ngdw8mk_nv3np240000gn/T/pip-req-build-lrvh1sf5
  Running command git clone --filter=blob:none --quiet https://github.com/matthias-k/DeepGaze /private/var/folders/kd/_s6tmg_d3ngdw8mk_nv3np240000gn/T/pip-req-build-lrvh1sf5
  Resolved https://github.com/matthias-k/DeepGaze to commit 874f12e1ee519860f49860638cf7f6375956d45a
  Preparing metadata (setup.py) ... done


In [3]:
!pip install lmdb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 169.5/169.5 kB 3.2 MB/s eta 0:00:00a 0:00:01


In [7]:
#!pip install git+https://github.com/matthias-k/pysaliency
!pip install git+https://github.com/Adrian-Samoticha/pysaliency.git

  Cloning https://github.com/Adrian-Samoticha/pysaliency.git to /private/var/folders/kd/_s6tmg_d3ngdw8mk_nv3np240000gn/T/pip-req-build-w5ol6hvd
  Running command git clone --filter=blob:none --quiet https://github.com/Adrian-Samoticha/pysaliency.git /private/var/folders/kd/_s6tmg_d3ngdw8mk_nv3np240000gn/T/pip-req-build-w5ol6hvd
  Resolved https://github.com/Adrian-Samoticha/pysaliency.git to commit f41ccfaba397eab02d0b228614f9b23d38df8c04
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 165.4/165.4 kB 3.3 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.3/313.3 kB 19.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 29.1 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.7/29.7 MB 28.5 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 28.8/28.8 MB 22.8 MB/s eta 0:00:0000:0100:01


In [8]:
# required to prevent:
# /opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
#   warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
!pip install scipy

In [9]:
from collections import OrderedDict
import os
from pathlib import Path
import shutil

from imageio.v3 import imread, imwrite
from PIL import Image
import pysaliency
from pysaliency.baseline_utils import BaselineModel, CrossvalidatedBaselineModel
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils import model_zoo

from tqdm import tqdm


from deepgaze_pytorch.layers import (
    Conv2dMultiInput,
    LayerNorm,
    LayerNormMultiInput,
    Bias,
    FlexibleScanpathHistoryEncoding
)

from deepgaze_pytorch.modules import DeepGazeIII, FeatureExtractor
from deepgaze_pytorch.features.densenet import RGBDenseNet201
from deepgaze_pytorch.data import ImageDataset, ImageDatasetSampler, FixationDataset, FixationMaskTransform
from deepgaze_pytorch.training import _train


ModuleNotFoundError: No module named 'sklearn'

In [7]:
def build_saliency_network(input_channels):
    return nn.Sequential(OrderedDict([
        ('layernorm0', LayerNorm(input_channels)),
        ('conv0', nn.Conv2d(input_channels, 8, (1, 1), bias=False)),
        ('bias0', Bias(8)),
        ('softplus0', nn.Softplus()),

        ('layernorm1', LayerNorm(8)),
        ('conv1', nn.Conv2d(8, 16, (1, 1), bias=False)),
        ('bias1', Bias(16)),
        ('softplus1', nn.Softplus()),

        ('layernorm2', LayerNorm(16)),
        ('conv2', nn.Conv2d(16, 1, (1, 1), bias=False)),
        ('bias2', Bias(1)),
        ('softplus2', nn.Softplus()),
    ]))


def build_scanpath_network():
    return nn.Sequential(OrderedDict([
        ('encoding0', FlexibleScanpathHistoryEncoding(in_fixations=4, channels_per_fixation=3, out_channels=128, kernel_size=[1, 1], bias=True)),
        ('softplus0', nn.Softplus()),

        ('layernorm1', LayerNorm(128)),
        ('conv1', nn.Conv2d(128, 16, (1, 1), bias=False)),
        ('bias1', Bias(16)),
        ('softplus1', nn.Softplus()),
    ]))


def build_fixation_selection_network(scanpath_features=16):
    return nn.Sequential(OrderedDict([
        ('layernorm0', LayerNormMultiInput([1, scanpath_features])),
        ('conv0', Conv2dMultiInput([1, scanpath_features], 128, (1, 1), bias=False)),
        ('bias0', Bias(128)),
        ('softplus0', nn.Softplus()),

        ('layernorm1', LayerNorm(128)),
        ('conv1', nn.Conv2d(128, 16, (1, 1), bias=False)),
        ('bias1', Bias(16)),
        ('softplus1', nn.Softplus()),

        ('conv2', nn.Conv2d(16, 1, (1, 1), bias=False)),
    ]))

In [8]:
def prepare_spatial_dataset(stimuli, fixations, centerbias, batch_size, path=None):
    if path is not None:
        path.mkdir(parents=True, exist_ok=True)
        lmdb_path = str(path)
    else:
        lmdb_path = None

    dataset = ImageDataset(
        stimuli=stimuli,
        fixations=fixations,
        centerbias_model=centerbias,
        transform=FixationMaskTransform(sparse=False),
        average='image',
        lmdb_path=lmdb_path,
    )

    loader = torch.utils.data.DataLoader(
        dataset,
        batch_sampler=ImageDatasetSampler(dataset, batch_size=batch_size),
        pin_memory=False,
        num_workers=0,
    )

    return loader

In [9]:
def prepare_scanpath_dataset(stimuli, fixations, centerbias, batch_size, path=None):
    if path is not None:
        path.mkdir(parents=True, exist_ok=True)
        lmdb_path = str(path)
    else:
        lmdb_path = None

    dataset = FixationDataset(
        stimuli=stimuli,
        fixations=fixations,
        centerbias_model=centerbias,
        included_fixations=[-1, -2, -3, -4],
        allow_missing_fixations=True,
        transform=FixationMaskTransform(sparse=False),
        average='image',
        lmdb_path=lmdb_path,
    )

    loader = torch.utils.data.DataLoader(
        dataset,
        batch_sampler=ImageDatasetSampler(dataset, batch_size=batch_size),
        pin_memory=False,
        num_workers=0,
    )

    return loader

In [10]:
dataset_directory = Path('pysaliency_datasets')
train_directory = Path('../train_deepgaze3')

print(train_directory)

../train_deepgaze3


In [4]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

NameError: name 'torch' is not defined

# Pretraining on SALICON

In [12]:
!pip install gdown

In [13]:
SALICON_train_stimuli, SALICON_train_fixations = pysaliency.get_SALICON_train(location=dataset_directory, max_samples=2)
SALICON_val_stimuli, SALICON_val_fixations = pysaliency.get_SALICON_val(location=dataset_directory, max_samples=2)

# parameters taken from an early fit for MIT1003. Since SALICON has many more fixations, the bandwidth won't be too small
SALICON_centerbias = BaselineModel(stimuli=SALICON_train_stimuli, fixations=SALICON_train_fixations, bandwidth=0.0217, eps=2e-13, caching=False)

# takes quite some time, feel free to set to zero
train_baseline_log_likelihood = 0.46408017115279737 #SALICON_centerbias.information_gain(SALICON_train_stimuli, SALICON_train_fixations, verbose=True, average='image')
val_baseline_log_likelihood = 0.4291592320821603 #SALICON_centerbias.information_gain(SALICON_val_stimuli, SALICON_val_fixations, verbose=True, average='image')

Downloading...
From (uriginal): https://drive.google.com/uc?id=1g8j-hTT-51IG1UFwP0xTGhLdgIUCW5e5
From (redirected): https://drive.google.com/uc?id=1g8j-hTT-51IG1UFwP0xTGhLdgIUCW5e5&confirm=t&uuid=6bbbcccb-2b5c-4f1d-b0f0-7058478fa455
To: /tmp/tmplaw5pe9f/stimuli.zip
100%|██████████| 2.27G/2.27G [00:24<00:00, 93.0MB/s]


Checking md5 sum...
Extracting stimuli


Downloading...
From (uriginal): https://drive.google.com/uc?id=1WVEiXba-I4GN33f0uUl4KhaN1rK7qzs6
From (redirected): https://drive.google.com/uc?id=1WVEiXba-I4GN33f0uUl4KhaN1rK7qzs6&confirm=t&uuid=d67241fd-4b55-41cd-a0af-8d19eb06badf
To: /tmp/tmp37lplyoi/fixations.zip
100%|██████████| 445M/445M [00:05<00:00, 87.7MB/s] 


Checking md5 sum...


100%|██████████| 2/2 [00:00<00:00, 269.72it/s]


In [14]:
model = DeepGazeIII(
    features=FeatureExtractor(RGBDenseNet201(), [
            '1.features.denseblock4.denselayer32.norm1',
            '1.features.denseblock4.denselayer32.conv1',
            '1.features.denseblock4.denselayer31.conv2',
        ]),
    saliency_network=build_saliency_network(2048),
    scanpath_network=None,
    fixation_selection_network=build_fixation_selection_network(scanpath_features=0),
    downsample=1.5,
    readout_factor=4,
    saliency_map_factor=4,
    included_fixations=[],
)

model = model.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
lr_scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[15, 30, 45, 60, 75, 90, 105, 120])

Downloading: "https://github.com/pytorch/vision/zipball/v0.6.0" to /root/.cache/torch/hub/v0.6.0.zip
/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DenseNet201_Weights.IMAGENET1K_V1`. You can also use `weights=DenseNet201_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/densenet201-c1103571.pth" to /root/.cache/torch/hub/checkpoints/densenet201-c1103571.pth
100%|██████████| 77.4M/77.4M [00:01<00:00, 78.1MB/s]


In [15]:
import gc

gc.collect()

train_loader = prepare_spatial_dataset(SALICON_train_stimuli, SALICON_train_fixations, SALICON_centerbias, batch_size=2, path=train_directory / 'lmdb_cache' / 'SALICON_train')
gc.collect()
validation_loader = prepare_spatial_dataset(SALICON_val_stimuli, SALICON_val_fixations, SALICON_centerbias, batch_size=2, path=train_directory / 'lmdb_cache' / 'SALICON_val')

Generate LMDB to ../train_deepgaze3/lmdb_cache/SALICON_train


100%|██████████| 2/2 [00:00<00:00,  9.73it/s]


Flushing database ...
Populating fixations cache


100%|█████████▉| 12940/12941 [00:00<00:00, 601930.79it/s]


Generate LMDB to ../train_deepgaze3/lmdb_cache/SALICON_val


100%|██████████| 2/2 [00:00<00:00, 65.89it/s]


Flushing database ...
Populating fixations cache


100%|█████████▉| 15523/15524 [00:00<00:00, 612442.79it/s]


In [16]:
_train(train_directory / 'pretraining',
    model,
    train_loader, train_baseline_log_likelihood,
    validation_loader, val_baseline_log_likelihood,
    optimizer, lr_scheduler,
    minimum_learning_rate=1e-7,
    device=device,
)


Using device cuda
validation metrics ['IG', 'LL', 'AUC', 'NSS']
Beginning training


LL 0.03047: 100%|██████████| 1/1 [00:04<00:00,  4.77s/it]


   epoch                  timestamp  learning_rate  loss  validation_IG  \
0      0 2023-12-30 10:26:06.098563          0.001   NaN      -0.398685   

   validation_LL  validation_AUC  validation_NSS  
0       0.030474        0.646169        0.612676  
validation_LL     0
validation_NSS    0
validation_AUC    0
validation_IG     0
dtype: int64


LL 0.06102: 100%|██████████| 1/1 [00:00<00:00,  5.25it/s]


   epoch                  timestamp  learning_rate      loss  validation_IG  \
0      0 2023-12-30 10:26:06.098563          0.001       NaN      -0.398685   
1      1 2023-12-30 10:26:07.640223          0.001 -0.685108      -0.368136   

   validation_LL  validation_AUC  validation_NSS  
0       0.030474        0.646169        0.612676  
1       0.061023        0.653873        0.636243  
validation_LL     1
validation_NSS    1
validation_AUC    1
validation_IG     1
dtype: int64


...

    epoch                  timestamp  learning_rate      loss  validation_IG  \
72     72 2023-12-30 10:26:49.958177   1.000000e-07 -0.869491      -0.537479   
73     73 2023-12-30 10:26:50.556567   1.000000e-07 -0.869491      -0.537481   

    validation_LL  validation_AUC  validation_NSS  
72      -0.108320        0.659483        0.428565  
73      -0.108321        0.659482        0.428563  
validation_LL     5
validation_NSS    4
validation_AUC    6
validation_IG     5
dtype: int64
removing ../train_deepgaze3/pretraining/step-0072.pth


LL -0.10832: 100%|██████████| 1/1 [00:00<00:00,  5.37it/s]


    epoch                  timestamp  learning_rate      loss  validation_IG  \
73     73 2023-12-30 10:26:50.556567   1.000000e-07 -0.869491      -0.537481   
74     74 2023-12-30 10:26:51.157662   1.000000e-07 -0.869493      -0.537480   

    validation_LL  validation_AUC  validation_NSS  
73      -0.108321        0.659482        0.428563  
74      -0.108321        0.659483        0.428562  
validation_LL     5
validation_NSS    4
validation_AUC    6
validation_IG     5
dtype: int64
removing ../train_deepgaze3/pretraining/step-0073.pth


LL -0.10832: 100%|██████████| 1/1 [00:00<00:00,  5.38it/s]


    epoch                  timestamp  learning_rate      loss  validation_IG  \
74     74 2023-12-30 10:26:51.157662   1.000000e-07 -0.869493      -0.537480   
75     75 2023-12-30 10:26:52.030540   1.000000e-07 -0.869494      -0.537481   

    validation_LL  validation_AUC  validation_NSS  
74      -0.108321        0.659483        0.428562  
75      -0.108322        0.659483        0.428560  
validation_LL     5
validation_NSS    4
validation_AUC    6
validation_IG     5
dtype: int64
removing ../train_deepgaze3/pretraining/step-0074.pth
removing ../train_deepgaze3/pretraining/step-0005.pth
removing ../train_deepgaze3/pretraining/step-0000.pth
removing ../train_deepgaze3/pretraining/step-0075.pth


# Preparing the MIT1003 dataset

In [17]:
# mit_stimuli_orig, mit_scanpaths_orig = pysaliency.external_datasets.mit.get_mit1003_with_initial_fixation(location=dataset_directory, replace_initial_invalid_fixations=True)

import pysaliency

source_directory = '/kaggle/input/mit1003-dataset/MIT1003 dataset'
destination_directory = '../datasets'

import shutil, errno
shutil.copytree(source_directory, destination_directory)

mit_stimuli_orig = pysaliency.read_hdf5(destination_directory + '/stimuli.hdf5')
mit_scanpaths_orig = pysaliency.read_hdf5(destination_directory + '/fixations.hdf5')

print(mit_stimuli_orig, mit_scanpaths_orig)

<pysaliency.datasets.FileStimuli object at 0x7935aa219540> <pysaliency.datasets.FixationTrains object at 0x7935a2f25b10>


In [18]:
!wget "http://people.csail.mit.edu/tjudd/WherePeopleLook/ALLSTIMULI.zip" -O "stimuli.zip"
!unzip "stimuli.zip" -d "../datasets"
!mv ../datasets/ALLSTIMULI ../datasets/stimuli

--2023-12-30 10:26:54--  http://people.csail.mit.edu/tjudd/WherePeopleLook/ALLSTIMULI.zip
Resolving people.csail.mit.edu (people.csail.mit.edu)... 128.30.2.133
Connecting to people.csail.mit.edu (people.csail.mit.edu)|128.30.2.133|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 235000497 (224M) [application/zip]
Saving to: ‘stimuli.zip’

stimuli.zip         100%[===================>] 224.11M  19.6MB/s    in 13s     

2023-12-30 10:27:08 (16.7 MB/s) - ‘stimuli.zip’ saved [235000497/235000497]

Archive:  stimuli.zip
   creating: ../datasets/ALLSTIMULI/
  inflating: ../datasets/ALLSTIMULI/.DS_Store  
   creating: ../datasets/__MACOSX/
   creating: ../datasets/__MACOSX/ALLSTIMULI/
  inflating: ../datasets/__MACOSX/ALLSTIMULI/._.DS_Store  
   creating: ../datasets/ALLSTIMULI/.svn/
  inflating: ../datasets/ALLSTIMULI/.svn/entries  
  inflating: ../datasets/ALLSTIMULI/.svn/format  
   creating: ../datasets/ALLSTIMULI/.svn/prop-base/
  inflating: ../datasets/ALLSTIMULI

In [19]:
def convert_stimulus(input_image):
    size = input_image.shape[0], input_image.shape[1]
    if size[0] < size[1]:
        new_size = 768, 1024
    else:
        new_size = 1024,768
    
    # pillow uses width, height
    new_size = tuple(list(new_size)[::-1])
    
    new_stimulus = np.array(Image.fromarray(input_image).resize(new_size, Image.BILINEAR))
    return new_stimulus

def convert_fixations(stimuli, fixations):
    new_fixations = fixations.copy()
    for n in tqdm(list(range(len(stimuli)))):
        stimulus = stimuli.stimuli[n]
        size = stimulus.shape[0], stimulus.shape[1]
        if size[0] < size[1]:
            new_size = 768, 1024
        else:
            new_size = 1024,768
        x_factor = new_size[1] / size[1]
        y_factor = new_size[0] / size[0]
        
        inds = new_fixations.n == n
        new_fixations.x[inds] *= x_factor
        new_fixations.y[inds] *= y_factor
        new_fixations.x_hist[inds] *= x_factor
        new_fixations.y_hist[inds] *= y_factor
    
    return new_fixations

def convert_fixation_trains(stimuli, fixations):
    train_xs = fixations.train_xs.copy()
    train_ys = fixations.train_ys.copy()
    
    for i in tqdm(range(len(train_xs))):
        n = fixations.train_ns[i]
        
        size = stimuli.shapes[n][0], stimuli.shapes[n][1]
        
        if size[0] < size[1]:
            new_size = 768, 1024
        else:
            new_size = 1024,768
        
        x_factor = new_size[1] / size[1]
        y_factor = new_size[0] / size[0]
        
        train_xs[i] *= x_factor
        train_ys[i] *= y_factor
        
    new_fixations = pysaliency.FixationTrains(
        train_xs = train_xs,
        train_ys = train_ys,
        train_ts = fixations.train_ts.copy(),
        train_ns = fixations.train_ns.copy(),
        train_subjects = fixations.train_subjects.copy(),
        attributes={key: getattr(fixations, key).copy() for key in fixations.__attributes__ if key not in ['subjects', 'scanpath_index']},
    )
    return new_fixations



def convert_stimuli(stimuli, new_location: Path):
    assert isinstance(stimuli, pysaliency.FileStimuli)
    new_stimuli_location = new_location / 'stimuli'
    new_stimuli_location.mkdir(parents=True, exist_ok=True)
    new_filenames = []
    for filename in tqdm(stimuli.filenames):
        stimulus = imread(filename)
        new_stimulus = convert_stimulus(stimulus)
        
        basename = os.path.basename(filename)
        new_filename = new_stimuli_location / basename
        if new_stimulus.size != stimulus.size:
            imwrite(new_filename, new_stimulus)
        else:
            #print("Keeping")
            shutil.copy(filename, new_filename)
        new_filenames.append(new_filename)
    return pysaliency.FileStimuli(new_filenames)

mit_scanpaths_twosize = convert_fixation_trains(mit_stimuli_orig, mit_scanpaths_orig)
mit_stimuli_twosize = convert_stimuli(mit_stimuli_orig, train_directory / 'MIT1003_twosize')

100%|██████████| 1003/1003 [00:13<00:00, 73.89it/s] 


In [20]:
# remove the initial forced fixation from the training data, it's only used for conditioning
mit_fixations_twosize = mit_scanpaths_twosize[mit_scanpaths_twosize.lengths > 0]

In [21]:
# parameters optimized on MIT1003 for maximum leave-one-image-out crossvalidation log-likelihood
MIT1003_centerbias = CrossvalidatedBaselineModel(
    mit_stimuli_twosize,
    mit_fixations_twosize,
    bandwidth=10**-1.6667673342543432,
    eps=10**-14.884189168516073,
    caching=False,
)

In [ ]:
for crossval_fold in range(10):
    MIT1003_stimuli_train, MIT1003_fixations_train = pysaliency.dataset_config.train_split(mit_stimuli_twosize, mit_fixations_twosize, crossval_folds=10, fold_no=crossval_fold)
    MIT1003_stimuli_val, MIT1003_fixations_val = pysaliency.dataset_config.validation_split(mit_stimuli_twosize, mit_fixations_twosize, crossval_folds=10, fold_no=crossval_fold)

    train_baseline_log_likelihood = MIT1003_centerbias.information_gain(MIT1003_stimuli_train, MIT1003_fixations_train, verbose=True, average='image')
    val_baseline_log_likelihood = MIT1003_centerbias.information_gain(MIT1003_stimuli_val, MIT1003_fixations_val, verbose=True, average='image')

    # finetune spatial model on MIT1003

    model = DeepGazeIII(
        features=FeatureExtractor(RGBDenseNet201(), [
                '1.features.denseblock4.denselayer32.norm1',
                '1.features.denseblock4.denselayer32.conv1',
                '1.features.denseblock4.denselayer31.conv2',
            ]),
        saliency_network=build_saliency_network(2048),
        scanpath_network=None,
        fixation_selection_network=build_fixation_selection_network(scanpath_features=0),
        downsample=2,
        readout_factor=4,
        saliency_map_factor=4,
        included_fixations=[],
    )

    model = model.to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    lr_scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[3, 6, 9, 12, 15, 18, 21, 24])

    train_loader = prepare_spatial_dataset(MIT1003_stimuli_train, MIT1003_fixations_train, MIT1003_centerbias, batch_size=4, path=train_directory / 'lmdb_cache' / f'MIT1003_train_spatial_{crossval_fold}')
    validation_loader = prepare_spatial_dataset(MIT1003_stimuli_val, MIT1003_fixations_val, MIT1003_centerbias, batch_size=4, path=train_directory / 'lmdb_cache' / f'MIT1003_val_spatial_{crossval_fold}')

    _train(train_directory / 'MIT1003_spatial' / f'crossval-10-{crossval_fold}',
        model,
        train_loader, train_baseline_log_likelihood,
        validation_loader, val_baseline_log_likelihood,
        optimizer, lr_scheduler,
        minimum_learning_rate=1e-7,
        device=device,
        startwith=train_directory / 'pretraining' / 'final.pth',
    )


    # Train scanpath model

    train_loader = prepare_scanpath_dataset(MIT1003_stimuli_train, MIT1003_fixations_train, MIT1003_centerbias, batch_size=4, path=train_directory / 'lmdb_cache' / f'MIT1003_train_scanpath_{crossval_fold}')
    validation_loader = prepare_scanpath_dataset(MIT1003_stimuli_val, MIT1003_fixations_val, MIT1003_centerbias, batch_size=4, path=train_directory / 'lmdb_cache' / f'MIT1003_val_scanpath_{crossval_fold}')

    # first train with partially frozen saliency network


    model = DeepGazeIII(
        features=FeatureExtractor(RGBDenseNet201(), [
                '1.features.denseblock4.denselayer32.norm1',
                '1.features.denseblock4.denselayer32.conv1',
                '1.features.denseblock4.denselayer31.conv2',
            ]),
        saliency_network=build_saliency_network(2048),
        scanpath_network=build_scanpath_network(),
        fixation_selection_network=build_fixation_selection_network(scanpath_features=16),
        downsample=2,
        readout_factor=4,
        saliency_map_factor=4,
        included_fixations=[-1, -2, -3, -4],
    )
    model = model.to(device)

    frozen_scopes = [
        "saliency_network.layernorm0",
        "saliency_network.conv0",
        "saliency_network.bias0",
        "saliency_network.layernorm1",
        "saliency_network.conv1",
        "saliency_network.bias1",
    ]

    for scope in frozen_scopes:
        for parameter_name, parameter in model.named_parameters():
            if parameter_name.startswith(scope):
                print("Fixating parameter", parameter_name)
                parameter.requires_grad = False


    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    lr_scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[10, 20, 30, 31, 32, 33, 34, 35])

    _train(train_directory / 'MIT1003_scanpath_partially_frozen_saliency_network' / f'crossval-10-{crossval_fold}',
        model,
        train_loader, train_baseline_log_likelihood,
        validation_loader, val_baseline_log_likelihood,
        optimizer, lr_scheduler,
        minimum_learning_rate=1e-7,
        device=device,
        startwith=train_directory / 'MIT1003_spatial' /  f'crossval-10-{crossval_fold}' / 'final.pth'
    )

    # Now finetune full scanpath model

    model = DeepGazeIII(
        features=FeatureExtractor(RGBDenseNet201(), [
                '1.features.denseblock4.denselayer32.norm1',
                '1.features.denseblock4.denselayer32.conv1',
                '1.features.denseblock4.denselayer31.conv2',
            ]),
        saliency_network=build_saliency_network(2048),
        scanpath_network=build_scanpath_network(),
        fixation_selection_network=build_fixation_selection_network(scanpath_features=16),
        downsample=2,
        readout_factor=4,
        saliency_map_factor=4,
        included_fixations=[-1, -2, -3, -4],
    )

    model = model.to(device)

    optimizer = torch.optim.Adam(model.parameters(), lr=0.00001)
    lr_scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[3, 6, 9, 12, 15, 18, 21, 24])

    _train(train_directory / 'MIT1003_scanpath' / f'crossval-10-{crossval_fold}',
        model,
        train_loader, train_baseline_log_likelihood,
        validation_loader, val_baseline_log_likelihood,
        optimizer, lr_scheduler,
        minimum_learning_rate=1e-7,
        device=device,
        startwith=train_directory / 'MIT1003_scanpath_partially_frozen_saliency_network' / f'crossval-10-{crossval_fold}' / 'final.pth'
    )


Using random shuffles for crossvalidation
Using random shuffles for crossvalidation


100%|██████████| 94/94 [00:15<00:00,  5.94it/s]
Using cache found in /root/.cache/torch/hub/pytorch_vision_v0.6.0
/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DenseNet201_Weights.IMAGENET1K_V1`. You can also use `weights=DenseNet201_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Generate LMDB to ../train_deepgaze3/lmdb_cache/MIT1003_train_spatial_0


100%|██████████| 808/808 [02:44<00:00,  4.93it/s]


Flushing database ...
Populating fixations cache


...

LL 2.39402: 100%|██████████| 2483/2483 [08:57<00:00,  4.62it/s]


   epoch                  timestamp  learning_rate      loss  validation_IG  \
1      1 2023-12-30 12:31:55.642803          0.001 -2.346558       1.438226   
2      2 2023-12-30 14:04:10.740719          0.001 -2.413532       1.482273   

   validation_LL  validation_AUC  validation_NSS  
1       2.349971        0.913652       10.019958  
2       2.394018        0.914819        8.057469  
validation_LL     2
validation_NSS    1
validation_AUC    2
validation_IG     2
dtype: int64
removing ../train_deepgaze3/MIT1003_scanpath_partially_frozen_saliency_network/crossval-10-0/step-0001.pth


LL 2.41901: 100%|██████████| 2483/2483 [09:03<00:00,  4.57it/s]


   epoch                  timestamp  learning_rate      loss  validation_IG  \
2      2 2023-12-30 14:04:10.740719          0.001 -2.413532       1.482273   
3      3 2023-12-30 15:36:54.390408          0.001 -2.436870       1.507265   

   validation_LL  validation_AUC  validation_NSS  
2       2.394018        0.914819        8.057469  
3       2.419011        0.916100        9.563972  
validation_LL     3
validation_NSS    1
validation_AUC    3
validation_IG     3
dtype: int64
removing ../train_deepgaze3/MIT1003_scanpath_partially_frozen_saliency_network/crossval-10-0/step-0002.pth


-2.45453:  69%|██████▊   | 14357/20930 [56:34<26:06,  4.20it/s] 